## Implementação de Sistema para Recomendação de Filmes baseado em Similaridade de Títulos

### 1° Passo - Mineração, Limpeza e Tratamento de Dados

Para adquirir os dados dos filmes, iremos minerar a API do site [TMDB](https://www.themoviedb.org/) e realizar transformações para termos um dataset unificado a partir de diferentes endpoints. 

Os endpoints utilizados foram:
* Dados resumidos dos filmes: https://api.themoviedb.org/3/discover/movie
* Dados detalhados dos filmes: https://api.themoviedb.org/3/movie/{movie_id}
* Dados dos atores e diretores: https://api.themoviedb.org/3/movie/{movie_id}/credits
* Dados das keywords: https://api.themoviedb.org/3/movie/{movie_id}/keywords
* Dados dos gêneros: https://api.themoviedb.org/3/genre/movie/list

Como filtros da mineração temos:
* Release_date >= 01/01/1990 e <= 31/07/2023
* Vote_count >= 100
* Original_language = en

In [1]:
# Bibliotecas
import requests
import json
import pandas as pd
import time
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
token = open('token.txt','r').read()

In [ ]:
# Chamada API para encontrar número de páginas
url = "https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&page=1&primary_release_date.gte=1990-01-01&primary_release_date.lte=2023-07-31&sort_by=primary_release_date.asc&vote_count.gte=100&with_original_language=en"

headers = {
    "accept": "application/json",
    "Authorization": token
}

response = requests.get(url, headers=headers)
response = json.loads(response.text)
numero_pagina = response.get('total_pages')

In [ ]:
# Chamada API para mineração dos dados resumidos dos filmes
dataset = pd.DataFrame()

for i in range (1, numero_pagina + 1):
    url = f"https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&page={i}&primary_release_date.gte=1990-01-01&primary_release_date.lte=2023-07-31&sort_by=primary_release_date.asc&vote_count.gte=100&with_original_language=en"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers)
    response = json.loads(response.text)
    response = response.get('results')
    response = pd.DataFrame(response)
    response.drop(['adult', 'backdrop_path', 'original_language', 'video'], axis = 1, inplace = True)
    dataset = pd.concat([dataset, response])
    time.sleep(1)
    if i % 10 == 0:
        print(i)

dataset.to_csv('dataset.csv', sep = ';', decimal = ',', index = False)

In [ ]:
# Chamada API para mineração dos dados detalhados dos filmes
dataset = pd.read_csv('dataset.csv', sep = ';')
dataset_detalhe = pd.DataFrame()

for i in dataset.index:
    movie_id = dataset.loc[i, 'id']

    url = f"https://api.themoviedb.org/3/movie/{movie_id}?language=en-US"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).text
    position = response.find('belongs_to_collection":null')
    if position == -1:
        position = response.find('belongs_to_collection')
        response = response[:position + 23] + '[' + response[position + 23:]
        position = response.find('budget')
        response = response[:position - 2] + ']' + response[position - 2:]
    response = json.loads(response)
    drop = ['adult', 'backdrop_path', 'genres', 'homepage', 'original_language', 'original_title', 'overview',\
            'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'runtime', 'spoken_languages',\
            'status', 'title', 'video', 'vote_average', 'vote_count']
    for key in drop:
        if key in response:
            response.pop(key)
    response = pd.DataFrame([response])
    dataset_detalhe = pd.concat([dataset_detalhe, response])
    time.sleep(0.5)
    if i % 100 == 0:
        print(i)

dataset_detalhe.to_csv('dataset_detalhe.csv', sep = ';', decimal = ',', index = False)

In [ ]:
# Chamada API para mineração dos dados dos atores dos filmes
dataset = pd.read_csv('dataset.csv', sep = ';')
dataset_elenco = pd.DataFrame()

for i in dataset.index:
    movie_id = dataset.loc[i, 'id']
    
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?language=en-US"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).text
    response = json.loads(response)
    response = response.get('cast')
    response = pd.DataFrame(response)
    if not response.empty:
        response = response[response['order'] <= 5]
        response.drop(['adult', 'known_for_department', 'popularity', 'profile_path', 'cast_id', 'character', 'credit_id', 'order'], axis = 1, inplace = True)
        response['movie_id'] = movie_id
    dataset_elenco = pd.concat([dataset_elenco, response])
    time.sleep(0.3)
    if i % 100 == 0:
        print(i)

dataset_elenco.to_csv('dataset_elenco.csv', sep = ';', decimal = ',', index = False)

In [ ]:
# Chamada API para mineração dos dados dos diretores dos filmes
dataset = pd.read_csv('dataset.csv', sep = ';')
dataset_diretor = pd.DataFrame()

for i in dataset.index:
    movie_id = dataset.loc[i, 'id']
    
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?language=en-US"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).text
    response = json.loads(response)
    response = response.get('crew')
    response = pd.DataFrame(response)
    if not response.empty:
        response = response[response['job'] == 'Director']
        response.drop(['adult', 'known_for_department', 'popularity', 'profile_path', 'credit_id', 'department'], axis = 1, inplace = True)
        response['movie_id'] = movie_id
    dataset_diretor = pd.concat([dataset_diretor, response])
    time.sleep(0.3)
    if i % 100 == 0:
        print(i)

dataset_diretor.to_csv('dataset_diretor.csv', sep = ';', decimal = ',', index = False)

In [ ]:
# Chamada API para mineração dos dados das keywords dos filmes
dataset = pd.read_csv('dataset_consolidado.csv', sep = ';')
dataset_keywords = pd.DataFrame()

for i in dataset.index:
    movie_id = dataset.loc[i, 'id']

    url = f"https://api.themoviedb.org/3/movie/{movie_id}/keywords"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).text
    response = json.loads(response)   
    response = response.get('keywords')
    response = pd.DataFrame(response)
    response['movie_id'] = movie_id
    dataset_keywords = pd.concat([dataset_keywords, response])
    time.sleep(0.3)
    if i % 100 == 0:
        print(i)

dataset_keywords.to_csv('dataset_keywords.csv', sep = ';', decimal = ',', index = False)

In [ ]:
# Unificação dos datasets
dataset = pd.read_csv('dataset_consolidado.csv', sep = ';')
dataset_detalhe = pd.read_csv('dataset_detalhe.csv', sep = ';')
dataset_elenco = pd.read_csv('dataset_elenco.csv', sep = ';')
dataset_diretor = pd.read_csv('dataset_diretor.csv', sep = ';')
dataset_keywords = pd.read_csv('dataset_keywords.csv', sep = ';')

dataset.drop('original_title',axis = 1, inplace = True)
dataset = pd.merge(dataset, dataset_detalhe, how = 'left', on = 'id')

dataset['poster_path'] = 'https://image.tmdb.org/t/p/original/' + dataset['poster_path']

for i in dataset.index:
    position = str(dataset.loc[i, 'belongs_to_collection']).find('[')
    if position == 0:
        position = dataset.loc[i, 'belongs_to_collection'].find('name')
        position2 = dataset.loc[i, 'belongs_to_collection'].find('poster_path')
        dataset.loc[i, 'belongs_to_collection'] = dataset.loc[i, 'belongs_to_collection'][position+8:position2-4]

dataset_elenco = dataset_elenco.groupby('movie_id')['id'].apply(list)
dataset = pd.merge(dataset, dataset_elenco, how = 'left', left_on='id', right_on='movie_id')
dataset = dataset.rename(columns={'id_x': 'id', 'id_y': 'cast_ids'})

dataset_diretor = dataset_diretor.groupby('movie_id')['id'].apply(list)
dataset = pd.merge(dataset, dataset_diretor, how = 'left', left_on='id', right_on='movie_id')
dataset = dataset.rename(columns={'id_x': 'id', 'id_y': 'director_ids'})

dataset_keywords = dataset_keywords.groupby('movie_id')['id'].apply(list)
dataset = pd.merge(dataset, dataset_keywords, how = 'left', left_on='id', right_on='movie_id')
dataset = dataset.rename(columns={'id_x': 'id', 'id_y': 'keywords_ids'})

dataset = dataset.reindex(columns=['id', 'title', 'belongs_to_collection', 'overview', 'tagline', 'poster_path', 'genre_ids', 'cast_ids', 'director_ids', 'keywords_ids', 'release_date', 'popularity', 'vote_average', 'vote_count', 'budget', 'revenue'])

dataset.to_csv('dataset_final.csv', sep = ';', index = False)

In [ ]:
# Chamada API para mineração dos genêros dos filmes
url = "https://api.themoviedb.org/3/genre/movie/list?language=en"

headers = {
    "accept": "application/json",
    "Authorization": token
}

response = requests.get(url, headers=headers)
response = json.loads(response.text).get('genres')
dataset = pd.DataFrame(response)
dataset.to_csv('generos.csv',sep=';',index=False)

### 2° Passo - Análise Exploratória dos Dados
Para realizar a análise foi criado um outro dataset para ser análisado em modelo dimensional utilizando a ferramenta Power BI, porém como passo anterior, tivemos que explodir as linhas que tinhamos em formato de lista para utilizar os relacionamentos com os datasets das dimensões (gêneros, atores, diretores, keywords).

In [ ]:
dataset = pd.read_csv('dataset_final.csv', sep = ';')
dataset['genre_ids'] = dataset['genre_ids'].apply(ast.literal_eval)
dataset = dataset.explode('genre_ids')
dataset['cast_ids'].fillna('[]', inplace= True)
dataset['cast_ids'] = dataset['cast_ids'].apply(ast.literal_eval)
dataset = dataset.explode('cast_ids')
dataset['director_ids'].fillna('[]', inplace= True)
dataset['director_ids'] = dataset['director_ids'].apply(ast.literal_eval)
dataset = dataset.explode('director_ids')
dataset['keywords_ids'].fillna('[]', inplace= True)
dataset['keywords_ids'] = dataset['keywords_ids'].apply(ast.literal_eval)
dataset = dataset.explode('keywords_ids')

dataset.to_csv('dataset_analise_exploratoria.csv', sep=';',index=False)

### 3° Passo - Desenvolvimento do Algoritmo de Recomendação
Para desenvolvimento do algoritmo de recomendação foi utilizado o conceito de similaridade de cossenos, a partir dos módulos TfidfVectorizer e linear_kernel da biblioteca scikit learn.

A similaridade de cossenos é uma técnica amplamente utilizada na análise de dados para medir a semelhança entre dois vetores em um espaço multidimensional. Ela é especialmente útil em tarefas como recuperação de informações, mineração de texto, recomendação de itens, classificação de documentos e muito mais. A similaridade de cossenos avalia o ângulo entre dois vetores, fornecendo uma medida de quão próximos ou semelhantes eles estão, independentemente do tamanho dos vetores.

1. Representação de Dados:
   - Para aplicar a similaridade de cossenos, primeiro você precisa representar seus dados como vetores num espaço multidimensional. Isso é comumente feito usando a representação de Bag of Words (BoW) ou TF-IDF (Term Frequency-Inverse Document Frequency) para documentos de texto, ou vetores de características para outros tipos de dados.

2. Ângulo entre Vetores:
   - A similaridade de cossenos se concentra na direção dos vetores, não no seu comprimento. Dois vetores são considerados mais semelhantes quanto menor for o ângulo entre eles. Se o ângulo for 0 graus, isso significa que os vetores são idênticos; se o ângulo for 90 graus, eles são ortogonais (não têm similaridade); e se o ângulo for 180 graus, eles são diametralmente opostos (máxima dissimilaridade).

3. Cálculo da Similaridade de Cossenos:
   - A fórmula para calcular a similaridade de cossenos entre dois vetores A e B é a seguinte:
   
   Similaridade(A, B) = (A . B) / (||A|| * ||B||)

   - Onde:
     - A . B é o produto escalar entre os vetores A e B.
     - ||A|| e ||B|| são as normas (comprimentos) dos vetores A e B.

4. Interpretação:
   - O resultado da similaridade de cossenos varia de -1 a 1.
   - Um valor de 1 indica que os vetores são idênticos em direção.
   - Um valor próximo a 0 indica que os vetores são ortogonais, ou seja, não possuem similaridade.
   - Um valor próximo a -1 indica que os vetores são diametralmente opostos em direção.

5. Aplicações:
   - A similaridade de cossenos é amplamente usada em sistemas de recomendação para encontrar itens semelhantes aos que um usuário já gostou.
   - É usada em pesquisa de informações para encontrar documentos ou páginas da web semelhantes a uma consulta.
   - É usada em mineração de texto para agrupar documentos semelhantes ou classificá-los em categorias.
   - Também é usada em análise de dados multidimensionais para encontrar a semelhança entre objetos em espaços de alta dimensão.

Em resumo, a similaridade de cossenos é uma técnica útil na análise de dados que permite medir a semelhança entre vetores em espaços multidimensionais, ignorando o comprimento dos vetores e enfocando apenas a direção. Ela é amplamente aplicável em uma variedade de cenários, desde análise de texto até sistemas de recomendação e muito mais.

In [2]:
dataset = pd.read_csv('dataset_final.csv', sep = ';')
dataset['belongs_to_collection'] = dataset['belongs_to_collection'].fillna('')
dataset['tagline'] = dataset['tagline'].fillna('')
dataset['overview'] = dataset['overview'].fillna('')
dataset['vetor_caracteristicas'] = dataset['overview'] + dataset['tagline'] + dataset['belongs_to_collection']

tf = TfidfVectorizer(analyzer = 'word')
tfidf_matrix = tf.fit_transform(dataset['vetor_caracteristicas'])

similaridade_cosseno = linear_kernel(tfidf_matrix, tfidf_matrix)

dataset = dataset.reset_index()
identificador = dataset['identificador']
titulos = pd.Series(dataset.index, index=dataset['identificador'])

def get_recommendations(titulo):
    index_recomendacao = titulos[titulo]
    similaridade_cosseno_pontuacao = list(enumerate(similaridade_cosseno[index_recomendacao]))
    similaridade_cosseno_pontuacao = sorted(similaridade_cosseno_pontuacao, key = lambda x: x[1], reverse = True)
    similaridade_cosseno_pontuacao = similaridade_cosseno_pontuacao[1:11]
    index_recomendacao = [i[0] for i in similaridade_cosseno_pontuacao]
    return identificador.iloc[index_recomendacao]

get_recommendations('2008 - The Dark Knight')

5301                      2012 - The Dark Knight Rises
9469       2021 - Batman: The Long Halloween, Part Two
9426       2021 - Batman: The Long Halloween, Part One
535                1993 - Batman: Mask of the Phantasm
5502    2013 - Batman: The Dark Knight Returns, Part 2
1751         2000 - Batman Beyond: Return of the Joker
9706                                 2022 - The Batman
7856                        2017 - Batman vs. Two-Face
2775                              2005 - Batman Begins
5336    2012 - Batman: The Dark Knight Returns, Part 1
Name: identificador, dtype: object

In [3]:
dataset['genres'] = dataset['genres'].fillna('')
dataset['cast'] = dataset['cast'].fillna('')
dataset['directors'] = dataset['directors'].fillna('')
dataset['keywords'] = dataset['keywords'].fillna('')

dataset['vetor_caracteristicas'] = dataset['overview'] + dataset['tagline'] + dataset['belongs_to_collection'].str.repeat(9) + \
                         dataset['genres'].str.repeat(6) + dataset['cast'] + dataset['directors'].str.repeat(3) + dataset['keywords']
                         
tf = TfidfVectorizer(analyzer = 'word')
tfidf_matrix = tf.fit_transform(dataset['vetor_caracteristicas'])

similaridade_cosseno = linear_kernel(tfidf_matrix, tfidf_matrix)

dataset = dataset.reset_index()
identificador = dataset['identificador']
titulos = pd.Series(dataset.index, index=dataset['identificador'])

def get_recommendations(titulo):
    index_recomendacao = titulos[titulo]
    similaridade_cosseno_pontuacao = list(enumerate(similaridade_cosseno[index_recomendacao]))
    similaridade_cosseno_pontuacao = sorted(similaridade_cosseno_pontuacao, key = lambda x: x[1], reverse = True)
    similaridade_cosseno_pontuacao = similaridade_cosseno_pontuacao[1:11]
    index_recomendacao = [i[0] for i in similaridade_cosseno_pontuacao]
    return identificador.iloc[index_recomendacao]

get_recommendations('2008 - The Dark Knight')

2775                               2005 - Batman Begins
5301                       2012 - The Dark Knight Rises
5336     2012 - Batman: The Dark Knight Returns, Part 1
5502     2013 - Batman: The Dark Knight Returns, Part 2
9942                     2022 - Detective Knight: Rogue
10022             2023 - Detective Knight: Independence
9706                                  2022 - The Batman
5622                                 2013 - Hummingbird
7521                     2017 - The Fate of the Furious
1682                              2000 - The Art of War
Name: identificador, dtype: object

### 4° Passo - Salvar recomendações para posterior exibição ao usuário final
Após o desenvolvimento do algoritmo e teste das recomendações, iremos gerar um novo dataset para salvarmos as recomendações para cada um dos filmes minerados na API. Com esses dados em mãos é possível desenvolver apresentações ao usuário final pelo próprio Python ou utilizando alguma ferramenta como Power BI.

In [4]:
identificadorId = dataset['id']

def get_recommendations(titulo):
    index_recomendacao = titulos[titulo]
    similaridade_cosseno_pontuacao = list(enumerate(similaridade_cosseno[index_recomendacao]))
    similaridade_cosseno_pontuacao = sorted(similaridade_cosseno_pontuacao, key = lambda x: x[1], reverse = True)
    similaridade_cosseno_pontuacao = similaridade_cosseno_pontuacao[1:11]
    index_recomendacao = [i[0] for i in similaridade_cosseno_pontuacao]
    return identificadorId.iloc[index_recomendacao], identificador.iloc[index_recomendacao]

dataset_recomendacoes = dataset[['id','identificador']]
dataset_recomendacoes = dataset_recomendacoes.assign(recomendacoesId = None)
dataset_recomendacoes = dataset_recomendacoes.assign(recomendacoes = None)
dataset_recomendacoes = dataset_recomendacoes.assign(recomendacoesRank = None)

for i in dataset_recomendacoes.index:
    recomendacao_id, recomendacao_filme = get_recommendations(dataset_recomendacoes.loc[i,'identificador'])
    dataset_recomendacoes.at[i,'recomendacoesId'] = list(recomendacao_id)
    dataset_recomendacoes.at[i,'recomendacoes'] = list(recomendacao_filme)
    dataset_recomendacoes.at[i,'recomendacoesRank'] = list(range(1, 11))

dataset_recomendacoes = dataset_recomendacoes.explode(['recomendacoesId','recomendacoes','recomendacoesRank'])

dataset_recomendacoes.to_csv('dataset_recomendacoes.csv', sep = ';', index = False)